<img src="https://github.com/FUlyankin/HSE_Data_Culture/blob/master/ML_for_marketing_2019/sems/sem9_clasters/embedding.gif?raw=true" height="500" width="500"> 

# <center> ML для маркетинга. <br>  <br> Cюжетная арка 3: сегментация </center>

В этом семестре мы с вами пройдём через несколько сюжетных арок. Третьей и заключительной такой аркой будет сегментация. 


#  Глава I (Люси заглядывает в платяной шкаф) 

## 1. Постановка задачи 

Представим, что международное круизное агентство "Carnival Cruise Line" решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой, и агентство продолжило с Вами сотрудничать.

Агентство крупное, и у него есть офисы по всему миру. Вблизи этих офисов оно и хочет разместить баннеры - легче договариваться и проверять результат. Также эти места должны хорошо просматриваться. Подумайте как бы вы решали эту задачу. Откуда бы вы взяли данные и какие? __Сначала подумайте, а после читайте блокнот дальше. Там ответы...__


## 2. Ход мыслей

Примерный ход ваших мыслей должен был выглядеть так: 

- Банеры. Нужно, чтобы их чаще смотрели. В точках, где они стоят нужны большие скопления людей. 
- Как найти большое скопление людей? По геолокации! Нужна база чекинов. 
- Тогда мы сможем кластеризовать чекины, найти самые популярные места в окрестнсоти каждого офиса и поставить там банеры. Задача будет решена, на банеры будут смотреть, а нам дадут денег. 


## 3. Учим машины и рисуем карты

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях - Foursqare. Часть открытых данных есть, например, на сайте [archive.org.](https://archive.org/details/201309_foursquare_dataset_umn) Нас будет интересовать файл `checkins.dat`. Подгрузим его и посмотрим, что лежит внутри. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv('checkins.csv')
print('Размер выборки:', df.shape)
df.head()

Внутри лежат  id наблюдений, id тех, кто зачекинился, время чекина, ещё какое-то id и координаты чекина. Именно координаты нас и интересуют. Заберём их в отдельную таблицу $X.$

In [ ]:
X = df[['longitude', 'latitude']]
X.head()

Как мы помним, 20 баннеров надо разместить около офисов компании. Если воспользоваться Google Maps, можно узнать координаты этих офисов.

```
33.751277, -118.188740 (Los Angeles)

25.867736, -80.324116 (Miami)

51.503016, -0.075479 (London)

52.378894, 4.885084 (Amsterdam)

39.366487, 117.036146 (Beijing)

-33.868457, 151.205134 (Sydney)
```

In [ ]:
# координаты офисов
ofices = np.array([[-118.188740, 33.751277],[-80.324116, 25.867736], [-0.075479, 51.503016],
                  [4.885084, 52.378894], [117.036146, 39.366487], [151.205134, -33.868457]])

Посмотрим как расположение офисов и чекинов выглядит на картинке и попробуем догадаться где какой материк.

In [ ]:
plt.figure(figsize=(16,8))
plt.scatter(X.longitude.values,X.latitude.values)
plt.scatter(ofices[:,0], ofices[:,1], color='red')

## 4. Кластеризация

Пришло время разбивать чекины на однородные группы. Будем делать это с помощью кластеризации. 

**Примечание**: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико, чтобы не повлиять на кластеризацию или сортировку.

**Примечание**: на 396632 строках, кластеризация будет работать долго. Для получения корректного ответа достаточно и 100000 (~2 минуты на "среднем" ноутбуке). Быть очень терпеливым не возбраняется - результат от этого только улучшится. Если 100000 компьютер тоже отказывается тянуть, возьмите более мелкую выборку. 

Сделаем сэмпл. 

In [ ]:
ind = np.random.choice(range(X.shape[0]), 100000)
X_sample = X.get_values()[ind]
X_sample.shape

__[1]__ Для начала попробуем сделать кластеризацию методом K-means на два кластера. 

In [ ]:
# Ваша кластеризация

In [ ]:
centers = #запишите сюда центры кластеров 
centers

In [ ]:
labels =   # запишите сюда лэйблы кластеров (единичка или нолик) 
labels[:10]

Запустите код ниже. Построится карта, где все точки будут покрашены в соотвествующие кластеры.

In [ ]:
from itertools import cycle

labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)

plt.figure(figsize=(16,8))

# Будем перебирать все цвета внутри вот этого цикла
colors = cycle('bgrcmykbgrcmykbgrcmykbgrcmyk')
for k, col in zip(range(n_clusters_), colors):
    my_members = labels == k
    center = centers[k]
    # Каждый кластер наносим на график своим цветом 
    plt.plot(X_sample[my_members,0],X_sample[my_members,1],col + '.' )

__[1]__ Попробуйте оценить k-means c 4, 5, 10, 20, 50 кластерами. Каждый раз просто перезапускайте код выше с новым числом $K$. Что происходит с точками на карте? Как именно чекины разбиваются на сегменты? Есть ли что-то плохое в таком поведении алгоритма для нашей задачи? 

__Ответ:__ 

Давайте попробуем теперь реализовать на наших данных немного другой алгоритм. Он называется DBSCAN. Пока что мы о нём ничего не знаем и понятия не имеем как он работает. 

__[1]__  Разберитесь как работает этот алгоритм и опишите интуицию которая за ним стоит. Математику опустите. В описании интуиции вам поможет [вот эта статья.](https://habr.com/ru/post/322034/)

__Ответ:__ 


__[1]__ Ну а теперь реализуйте алгоритм на наших данных. Написать несколько волшебных строк для этого вам поможет [документация к sklearn.](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) Обоснуйте свой выбор параметров алгоритма (параметры `eps` и `min_samples`).

In [ ]:
# Ваша кластеризация :) 

In [ ]:
labels = # запишите сюда лэйблы кластеров 

Давайте посмотрим что у нас получится на карте. 

In [ ]:
from itertools import cycle

labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)

plt.figure(figsize=(16,8))

# Будем перебирать все цвета внутри вот этого цикла
colors = cycle('bgrcmykbgrcmykbgrcmykbgrcmyk')
for k, col in zip(range(n_clusters_), colors):
    my_members = labels == k
    # Каждый кластер наносим на график своим цветом 
    plt.plot(X_sample[my_members,0],X_sample[my_members,1],col + '.' )

__[1]__  Сколько получилось кластеров? 

In [ ]:
# выведите на экран ответ (прям кодом!)

Что такое $-1$ кластер? Какой в нём смысл (чтобы ответить на этот вопрос надо было почитать про алгоритм...) 

__Ответ:__ 

Сколько в него попало точек? 

In [ ]:
# выведите на экран ответ (прям кодом!)

__[1]__ Как думаете, подходит ли DBSCAN для решения нашей задачи? У него явно есть какие-то минусы, которые конфликтуют с бизнец-целью, поставленной перед нами. Что это за минусы? 

__Ответ:__ 

Это всё, конечно, круто, но баннеры имеют сравнительно небольшую площадь действия. Кластеры, которые мы получили выше оказались какими-то слишком большими. Хочется найти какой-то алгоритм, который позволяет ограничить размер кластера. Он должен не зависеть от их числа.  

И такой алгоритм, действительно есть! Он называется `MeanShift`. Его описание при желании можно посмотреть в [sklearn user guide.](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.MeanShift.html)  В этом алгоритме можно искуственно ограничить радиус каждого кластера с помощью специальной опции `bandwidth`. Укажем `bandwidth=0.1`, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.


__[2]__  Подгрузете `MeanShift` из `sklearn` и запустите его на наших данных. Установите параметры `bin_seeding=True` и `min_bin_freq=15`. За что отвечают эти параметры? (ответ ищите в документации)

__Ответ:__ 

In [ ]:
# Ваша кластеризация

Сколько всего кластеров выделила модель? 

In [ ]:
# выведите на экран ответ (прям кодом!)

In [ ]:
labels =    # запишите сюда лэйблы кластеров 
centers =   # запишите сюда координаты центров кластеров

Посмотрим на то, как наши кластеры выглядят на карте мира!

In [ ]:
from itertools import cycle

labels = clt.labels_
labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)

plt.figure(figsize=(16,8))

# Будем перебирать все цвета внутри вот этого цикла
colors = cycle('bgrcmykbgrcmykbgrcmykbgrcmyk')
for k, col in zip(range(n_clusters_), colors):
    my_members = labels == k
    center = centers[k]
    # Каждый кластер наносим на график своим цветом 
    plt.plot(X_sample[my_members,0],X_sample[my_members,1],col + '.' )

Каждый кластер нанесён на карту своим цветом. Красота! Давайте построим точно такую же картинку, но ещё и с центрами кластеров.

In [ ]:
plt.figure(figsize=(16,8))

colors = cycle('bgrcmykbgrcmykbgrcmykbgrcmyk')
for k, col in zip(range(n_clusters_), colors):
    my_members = labels == k
    center = centers[k]
    
    # Каждый кластер наносим на график своим цветом 
    plt.plot(X_sample[my_members,0],X_sample[my_members,1],col + '.' )
    
    # Выделяем у кластеров центры 
    plt.plot(center[0], center[1], 'o', markerfacecolor=col, markeredgecolor='k', markersize=14)

Мы немного отвлеклись на картинки. Давайте вернёмся к нашей задаче. Мы должны выбрать 20 самых близких к офисам точек. 

__[2]__ Давайте найдём такие точки. Для этого отсортируем центры кластеров по расстояниям до каждого офиса. 

In [ ]:
results = [ ]
for item in ofices:
    for jtem in centers:
        
        ###############
        cur_rast = # посчитайте евклидово расстояние от текущего центра jtem до текущего офиса item
        ###############
        
        trio = (cur_rast, jtem, item)
        results.append(trio)

In [ ]:
# внутри results лежат тройки (расстояние, координаты центра кластера, координаты офиса)
results[0]  

In [ ]:
# всего у нас есть вот столько вариантов расположить рекламные щиты
len(results)

Возьмём 20 самых близких офисов.

In [ ]:
results.sort()         # отсортировали по расстоянию
best_20 = results[:20] # взяли 20 самых близких
best_20[:5]

In [ ]:
# Распакуем координаты для строительства картинки 
coord = np.array([[item[1][0],item[1][1]] for item in best_20])

In [ ]:
coord[:5] # координаты распаковались :) 

Строим ещё картинку! Красным нанесём офисы, лиловым точки для рекламы. Из-за больших масштабов, точки наложились друг на друга.

In [ ]:
plt.figure(figsize=(16,8))
plt.scatter(X.longitude.values,X.latitude.values, alpha=0.1)  # Наносим все чекины
plt.scatter(ofices[:,0], ofices[:,1], color='red')            # Наносим офисы 
plt.scatter(coord[:,0], coord[:,1], color='magenta')          # Наносим точки для рекламы 

Посмотрим в окрестностях какого офиса оказалась большая часть баннеров.

In [ ]:
l = 5
с_long = -118.18874
c_lat = 33.751277

X_int = X[(X.longitude > с_long - l)&(X.longitude < с_long + l)& 
          (X.latitude >  - l)&(X.latitude < c_lat + l)]
    
coord_int = coord[(coord[:,0] > с_long - l)&(coord[:,0] < с_long + l)& 
                  (coord[:,1] > c_lat - l)&(coord[:,1] < c_lat + l)]    


ofices_int = ofices[(ofices[:,0] > с_long - l)&(ofices[:,0] < с_long + l)& 
                    (ofices[:,1] > c_lat - l)&(ofices[:,1] < c_lat + l)] 

plt.figure(figsize=(16,8))
plt.scatter(X_int.longitude.values,X_int.latitude.values, alpha=0.1)  # Наносим все чекины
plt.scatter(ofices_int[:,0], ofices_int[:,1], color='red')            # Наносим офисы 
plt.scatter(coord_int[:,0], coord_int[:,1], color='magenta')          # Наносим точки для рекламы 

Это Калифорния! 

__[дополнительные 5 баллов]__ Компании безумно понравился наш алгоритм! Она просит нас внести небольшую правку. Ребята согласны пожертвовать близостью щита к офису в пользу большего числа чекинов, то есть в пользу большей просматриваемости. 

Найдите $20$ самых близких к офисам точек с более, чем $100$ чекинами. Нарисуйте их на карте. 

In [ ]:
# Ваш код

In [ ]:
# И ещё немного вашего кода 

## 4. Подводим итоги

Итак, мы только что помогли выбрать круизному агенству просматриваемые и близкие к офисам точки для рекламы. В ходе решения задачи: 

- Мы выслушали заказчика и поняли какие есть ограничения.
- Мы придумали откуда взять данные и как по ним найти популярные места. 
- Мы нашли подходящий алгоритм кластеризации и обучили модель.
- Мы подыскали самые просматриваемые локации в окрестности офисов. 
- Мы клёвые и заслужили гору бабла. 

Осталось только нарисовать красивую презентацию и показать это добро "Carnival Cruise Line". 